In [24]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from collections import defaultdict

import networkx as nx

In [38]:
DATA_PATH = "data/"

train_orig =  pd.read_csv(DATA_PATH + 'train.csv', header=0)
test_orig =  pd.read_csv(DATA_PATH + 'test.csv', header=0)

In [39]:
# "id","qid1","qid2","question1","question2","is_duplicate"
df_id1 = train_orig[["qid1", "question1"]].drop_duplicates(keep="first").copy().reset_index(drop=True)
df_id2 = train_orig[["qid2", "question2"]].drop_duplicates(keep="first").copy().reset_index(drop=True)

df_id1.columns = ["qid", "question"]
df_id2.columns = ["qid", "question"]

print(df_id1.shape, df_id2.shape)

df_id = pd.concat([df_id1, df_id2]).drop_duplicates(keep="first").reset_index(drop=True)
print(df_id1.shape, df_id2.shape, df_id.shape)

(290654, 2) (299364, 2)
(290654, 2) (299364, 2) (537933, 2)


In [40]:
import csv
dict_questions = df_id.set_index('question').to_dict()
dict_questions = dict_questions["qid"]

new_id = 538000 # df_id["qid"].max() ==> 537933

def get_id(question):
    global dict_questions 
    global new_id 
    
    if question in dict_questions:
        return dict_questions[question]
    else:
        new_id += 1
        dict_questions[question] = new_id
        return new_id
    
rows = []
max_lines = 10
if True:
    with open('data/test.csv', 'r', encoding="utf8") as infile:
        reader = csv.reader(infile, delimiter=",")
        header = next(reader)
        header.append('qid1')
        header.append('qid2')
        
        if True:
            print(header)
            pos, max_lines = 0, 10*1000*1000
            for row in reader:
                # "test_id","question1","question2"
                question1 = row[1]
                question2 = row[2]

                qid1 = get_id(question1)
                qid2 = get_id(question2)
                row.append(qid1)
                row.append(qid2)

                pos += 1
                if pos >= max_lines:
                    break
                rows.append(row)

['test_id', 'question1', 'question2', 'qid1', 'qid2']


In [41]:
test_orig = pd.DataFrame(rows, columns=['test_id', 'question1', 'question2', 'qid1', 'qid2'])

In [54]:
test_orig.to_csv('data/questions_with_id/test_questions_with_id.csv')
train_orig.to_csv('data/questions_with_id/train_questions_with_id.csv')

In [43]:
train_orig.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [44]:
test_orig.head()

,test_id,question1,question2,qid1,qid2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...,538001,538002
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?,538003,538004
2,2,What but is the best way to send money from Ch...,What you send money to China?,538005,538006
3,3,Which food not emulsifiers?,What foods fibre?,538007,538008
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?,538009,538010


In [47]:
df_train = pd.read_csv("data/questions_with_id/test_questions_with_id.csv", usecols=["qid1", "qid2"])

df_test = pd.read_csv("data/questions_with_id/train_questions_with_id.csv", usecols=["qid1", "qid2"])

df_all = pd.concat([df_train, df_test])

print("df_all.shape:", df_all.shape) # df_all.shape: (2750086, 2)

df = df_all

g = nx.Graph()

g.add_nodes_from(df.qid1)

edges = list(df[['qid1', 'qid2']].to_records(index=False))

g.add_edges_from(edges)

g.remove_edges_from(g.selfloop_edges())

print(len(set(df.qid1)), g.number_of_nodes()) # 4789604

print(len(df), g.number_of_edges()) # 2743365 (after self-edges)

df_output = pd.DataFrame(data=g.nodes(), columns=["qid"])

print("df_output.shape:", df_output.shape)

NB_CORES = 20

for k in range(2, NB_CORES + 1):

    fieldname = "kcore{}".format(k)

    print("fieldname = ", fieldname)

    ck = nx.k_core(g, k=k).nodes()

    print("len(ck) = ", len(ck))

    df_output[fieldname] = 0

    df_output.ix[df_output.qid.isin(ck), fieldname] = k

df_output.to_csv("data/kcores/question_kcores.csv", index=None)

df_all.shape: (2750086, 2)
2454493 4789604
2750086 2743365
df_output.shape: (4789604, 1)
fieldname =  kcore2
len(ck) =  104340


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:43: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


fieldname =  kcore3
len(ck) =  37019
fieldname =  kcore4
len(ck) =  20580
fieldname =  kcore5
len(ck) =  13552
fieldname =  kcore6
len(ck) =  9949
fieldname =  kcore7
len(ck) =  7674
fieldname =  kcore8
len(ck) =  6156
fieldname =  kcore9
len(ck) =  5124
fieldname =  kcore10
len(ck) =  4190
fieldname =  kcore11
len(ck) =  3606
fieldname =  kcore12
len(ck) =  2979
fieldname =  kcore13
len(ck) =  2529
fieldname =  kcore14
len(ck) =  2167
fieldname =  kcore15
len(ck) =  1807
fieldname =  kcore16
len(ck) =  1633
fieldname =  kcore17
len(ck) =  1523
fieldname =  kcore18
len(ck) =  1270
fieldname =  kcore19
len(ck) =  1125
fieldname =  kcore20
len(ck) =  1024


In [48]:
df_cores = pd.read_csv("data/kcores/question_kcores.csv", index_col="qid")

df_cores.index.names = ["qid"]

df_cores['max_kcore'] = df_cores.apply(lambda row: max(row), axis=1)

df_cores[['max_kcore']].to_csv("question_max_kcores.csv") # with index

/usr/local/lib/python3.6/site-packages/numpy/lib/arraysetops.py:395: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [56]:
cores_dict = pd.read_csv("data/kcores/question_max_kcores.csv", index_col="qid").to_dict()["max_kcore"]
df_train = pd.read_csv('data/questions_with_id/train_questions_with_id.csv')
df_test = pd.read_csv('data/questions_with_id/test_questions_with_id.csv')

def gen_qid1_max_kcore(row):
    return cores_dict[row["qid1"]]
def gen_qid2_max_kcore(row):
    return cores_dict[row["qid2"]]

#def gen_max_kcore(row):
#    return max(row["qid1_max_kcore"], row["qid2_max_kcore"])

df_train["qid1_max_kcore"] = df_train.apply(gen_qid1_max_kcore, axis=1)
df_test["qid1_max_kcore"] = df_test.apply(gen_qid1_max_kcore, axis=1)
df_train["qid2_max_kcore"] = df_train.apply(gen_qid2_max_kcore, axis=1)
df_test["qid2_max_kcore"] = df_test.apply(gen_qid2_max_kcore, axis=1)

/usr/local/lib/python3.6/site-packages/numpy/lib/arraysetops.py:395: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [57]:
df_train.to_csv('data/kcores/train_max_cores.csv')
df_test.to_csv('data/kcores/test_max_cores.csv')